In [1]:
import numpy as np
import data

In [2]:
train = data.train
test = data.test

In [3]:
class MatrixFactorization():
    
    def __init__(self, train, test, k, learning_rate, epochs, verbose = False):
        """
        param R : Rating Matrix
        param sd : standard deviation of U and V
        param k : latent parameter
        param learning_rate : alpha on weight update
        param epochs : training epochs
        param verbose : print status
        """
        
        self._R = train
        self._test = test
        self._I = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(train), dtype = np.float64) # indicator matrix
        self._n_user_rated = np.sum(self._I, axis = 1)
        self._n_item_rated = np.sum(self._I, axis = 0)
        self._num_users, self._num_items = train.shape
        # sigma?
        self._lambda_U = 0.01
        self._lambda_V = 0.01
        self._k = k
        self._learning_rate = learning_rate
        self._epochs = epochs
        self._verbose = verbose
        
        
    def fit(self):
        """
        training Matrix Factorization : update matrix latent weight and bias
        """
        # init latent features
        self._U = np.random.normal(0, 0.1, size=(self._num_users, self._k))
        self._V = np.random.normal(0, 0.1, size=(self._num_items, self._k))
        
        self._training_process = []
        for epoch in range(self._epochs):
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0 :
                        self.gradient_descent(i, j, self._R[i, j])
                        
            train_cost, test_cost = self.cost()
            self._training_process.append((epoch, train_cost, test_cost))
            
            if self._verbose == True and ((epoch + 1) % 10 == 0 ):
                print("Iteration : %d, train_cost = %.4f, test_cost = %.4f" % (epoch+1, train_cost, test_cost))
                
                
    def cost(self):
        """
        compute RMSE
        """
        xi, yi = self._R.nonzero() # 0 이 아닌 값의 index 반환
        test_x, test_y = self._test.nonzero()
        predicted = self.get_complete_matrix()
        cost_train = 0
        cost_test = 0
        
        for x, y in zip(xi, yi):
            cost_train += pow(self._R[x, y] - predicted[x, y], 2)
        
        for x, y in zip(test_x, test_y):
            cost_test += pow(self._test[x, y] - predicted[x, y], 2)
        
        return np.sqrt(cost_train/len(xi)), np.sqrt(cost_test/len(test_x))
    
    
    def gradient_descent(self, i, j, rating):
        """
        gradient descent function
        param i : user index
        param j : item index
        param rating : rating of (i, j)
        """
        prediction = self.get_prediction(i, j)
        error = rating - prediction
        
        # self._U[i, :] += self._learning_rate * ( error * self._V[j, :] - self._lambda_U / self._n_user_rated[i] * self._U[i, :])
        # self._V[j, :] += self._learning_rate * ( error * self._U[i, :] - self._lambda_V / self._n_item_rated[j] * self._V[j, :])
        
        self._U[i, :] += self._learning_rate * ( error * self._V[j, :] - self._lambda_U * self._U[i, :])
        self._V[j, :] += self._learning_rate * ( error * self._U[i, :] - self._lambda_V * self._V[j, :])

    
    def get_prediction(self, i, j):
        """
        get predicted rating by user i on item j
        """
        
        return self._U[i, :].dot(self._V[j, :].T)
    
    
    def get_complete_matrix(self):
        """
        compute complete matrix
        """
        
        return self._U.dot(self._V.T)
    
    
    def print_results(self):
        """
        print fit results
        """

        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][2])

In [4]:
if __name__ == "__main__":
    
    np.random.seed(7)
    
    np.seterr(all="warn")
    
    factorizer = MatrixFactorization(train, test, k=40, learning_rate=0.001, epochs=100, verbose=True)
    # regression parameter 2개
    factorizer.fit()
    factorizer.print_results()

Iteration : 10, train_cost = 2.1955, test_cost = 2.5017
Iteration : 20, train_cost = 1.1631, test_cost = 1.3211
Iteration : 30, train_cost = 0.9905, test_cost = 1.1158
Iteration : 40, train_cost = 0.9259, test_cost = 1.0449
Iteration : 50, train_cost = 0.8861, test_cost = 1.0114
Iteration : 60, train_cost = 0.8539, test_cost = 0.9922
Iteration : 70, train_cost = 0.8246, test_cost = 0.9799
Iteration : 80, train_cost = 0.7967, test_cost = 0.9720
Iteration : 90, train_cost = 0.7698, test_cost = 0.9672
Iteration : 100, train_cost = 0.7438, test_cost = 0.9647
Final R matrix:
[[3.77101393 3.28140676 3.33395649 ... 0.21264457 0.74280367 1.12970716]
 [4.01626753 2.9738343  2.81161759 ... 0.39094047 0.58769299 0.94336919]
 [3.17926172 2.45748069 2.39312677 ... 0.36813159 0.36630343 0.75542446]
 ...
 [4.42027193 3.30849612 3.15116142 ... 0.48317852 0.58691845 0.98679374]
 [4.46309119 3.71424653 3.10667433 ... 0.37567089 0.65637171 0.85270466]
 [3.54787446 3.56050093 2.93180972 ... 0.42537956 0.4